In [ ]:
%env GOOGLE_APPLICATION_CREDENTIALS=

In [ ]:
from google.cloud import datacatalog_v1

In [ ]:
datacatalog = datacatalog_v1.DataCatalogClient()

In [ ]:
resource_name = '//bigquery.googleapis.com/projects/{}/datasets/{}'\
    .format("sarun-project", "dsi_dataset")

print(datacatalog.lookup_entry(request={'linked_resource': resource_name}))

In [ ]:
project_id = 'sarun-project'
location = 'us'
dataset_id = 'dsi_dataset'
table_id = 'taxi_trips'

In [ ]:
tag_template = datacatalog_v1.types.TagTemplate()

tag_template.display_name = 'Python Tag Template'

tag_template.fields['code'] = datacatalog_v1.types.TagTemplateField()
tag_template.fields['code'].display_name = 'Code that produce the tag template'
tag_template.fields['code'].type_.primitive_type = datacatalog_v1.types.FieldType.PrimitiveType.STRING

tag_template.fields['num_rows'] = datacatalog_v1.types.TagTemplateField()
tag_template.fields['num_rows'].display_name = 'Number of rows in data asset'
tag_template.fields['num_rows'].type_.primitive_type = datacatalog_v1.types.FieldType.PrimitiveType.DOUBLE

tag_template.fields['quality'] = datacatalog_v1.types.TagTemplateField()
tag_template.fields['quality'].display_name = 'Data quality'

for display_name in ['GOOD', 'NORMAL', 'POOR']:
    enum_value = datacatalog_v1.types.FieldType.EnumType.EnumValue(display_name = display_name)
    tag_template.fields['quality'].type_.enum_type.allowed_values.append(enum_value)

expected_template_name = datacatalog_v1.DataCatalogClient\
    .tag_template_path(project_id, location, 'python_tag_template')

In [ ]:
try:
    tag_template = datacatalog.create_tag_template(
        parent='projects/{}/locations/{}'.format(project_id, location),
        tag_template_id='python_tag_template',
        tag_template=tag_template)
    print('Created template: {}'.format(tag_template.name))
except OSError as e:
    print('Cannot create template: {}'.format(expected_template_name))
    print('{}'.format(e))

In [ ]:
resource_name = '//bigquery.googleapis.com/projects/{}' \
                '/datasets/{}/tables/{}'.format(project_id, dataset_id, table_id)
table_entry = datacatalog.lookup_entry(request={"linked_resource": resource_name})

In [ ]:
tag = datacatalog_v1.types.Tag()

tag.template = tag_template.name
tag.name="python_tag"

tag.fields['code'] = datacatalog_v1.types.TagField()
tag.fields['code'].string_value = 'create_metadata.ipynb'

tag.fields['num_rows'] = datacatalog_v1.types.TagField()
tag.fields['num_rows'].double_value = 16385532

tag.fields['quality'] = datacatalog_v1.types.TagField()
tag.fields['quality'].enum_value.display_name = 'NORMAL'

In [ ]:
tag = datacatalog.create_tag(parent=table_entry.name, tag=tag)
print('Created tag: {}'.format(tag.name))